<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/ibm_granite_granite_3_0_8b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%sh
pip install torch torchvision torchaudio
pip install accelerate==0.34.0
pip install --upgrade "transformers>=4.45"
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "auto"
model_path = "ibm-granite/granite-3.0-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map=device)
model.eval()



tokenizer_config.json:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49155, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=12800, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=12800, bias=False)
          (down_proj): Linear8bitLt(in_features=12800, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((4096,), eps=1e-0

In [4]:
# change input text as desired
chat = [
    { "role": "user", "content": "へいへいへーい！俺を驚かすようなビックリする事とそれを使った一流のジョークを頼むぜ、ビッグブラザー！" },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to("cuda")
# generate output tokens
output = model.generate(**input_tokens,
                        max_new_tokens=300)
# decode output tokens into text
output = tokenizer.batch_decode(output)
model_marker = "model\n"
model_response = output[0].split(model_marker)[-1].strip()
print(model_response)


<|start_of_role|>user<|end_of_role|>へいへいへーい！俺を驚かすようなビックリする事とそれを使った一流のジョークを頼むぜ、ビッグブラザー！<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>こんにちは！私はあなたの助けに立ちます。私は人工知能で、あなたの質問に答え、ジョークを言うことができます。何かありますか？<|end_of_text|>
